In [ ]:
%pip install tsts==1.1.3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tsts.models import Transformer
from tsts.utils import TimeSeriesTensor

# Carregar os dados
metric_df = pd.read_pickle("../data/ts.pkl")
ts = metric_df["value"].astype(float).resample("min").mean()

# Dividir os dados em conjuntos de treinamento e teste
train_size = int(len(ts) * 0.8)
train_ts, test_ts = ts[:train_size], ts[train_size:]

# Normalizar os dados de treinamento e teste
scaler = MinMaxScaler()
train_ts = scaler.fit_transform(train_ts.values.reshape(-1, 1)).flatten()
test_ts = scaler.transform(test_ts.values.reshape(-1, 1)).flatten()

# Preparar os dados de treinamento e teste em janelas de tempo
lookback = 24
horizon = 1
train_data = TimeSeriesTensor(train_ts, target=train_ts, lookback=lookback, horizon=horizon)
test_data = TimeSeriesTensor(test_ts, target=test_ts, lookback=lookback, horizon=horizon)

# Definir a arquitetura do modelo Transformer
model = Transformer(
    input_size=lookback,
    output_size=horizon,
    d_model=64,
    nhead=4,
    num_encoder_layers=2,
    num_decoder_layers=2,
    dim_feedforward=128,
    dropout=0.1,
    activation="relu",
    batch_first=True,
)

# Treinar o modelo com os dados de treinamento
model.fit(train_data, epochs=10, batch_size=32)

# Avaliar o modelo com os dados de teste
y_pred = model.predict(test_data)
y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
test_ts = scaler.inverse_transform(test_ts.reshape(-1, 1)).flatten()
mae = np.mean(np.abs(test_ts[lookback:] - y_pred))
print(f"MAE: {mae}")
